In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('mqoau.csv', dtype={'Mode bits': str})

In [3]:
print(df.columns.tolist())

['Original_Query', 'Query Type', 'Annotated Answer', 'Query_Type', 'LLM Execution sequence', 'Mode bits', 'Execution mode', 'assignment', 'struct_id', 'plan', 'Final_Output', 'step_logs', 'total_cost', 'total_energy', 'total_latency', 'blending_cost', 'blending_energy', 'blending_latency', 'Total_Cost', 'Total_Energy', 'Total_Latency', 'Quality_of_Answer', 'domain']


In [4]:
print(df['struct_id'].unique())

[  0   1   5  41 801  50 802 836   6  52 840]


In [5]:
print(df['assignment'].unique())

['(0,)' '(1,)' '(2,)' ... '(3, 3, 5)' '(3, 4, 5)' '(4, 4, 5)']


In [6]:
unique_assignments = df['assignment'].unique()
print(unique_assignments)
print("Total unique assignments:", len(unique_assignments))

['(0,)' '(1,)' '(2,)' ... '(3, 3, 5)' '(3, 4, 5)' '(4, 4, 5)']
Total unique assignments: 1545


In [7]:
agg_df = df.groupby(['assignment', 'struct_id', 'Query_Type'])[['Total_Cost', 'Total_Energy', 'Total_Latency', 'Quality_of_Answer']].mean().reset_index()
print(agg_df.head())

  assignment  struct_id Query_Type  Total_Cost  Total_Energy  Total_Latency  \
0     (0, 0)          1        Art    0.000009     32.169121      49.514738   
1     (0, 0)          1  Geography    0.000009     30.835237      47.461623   
2     (0, 0)          1    History    0.000009     30.942640      47.626939   
3     (0, 0)          1      Music    0.000008     29.705766      45.723141   
4     (0, 0)          1      Other    0.000009     30.984216      47.690932   

   Quality_of_Answer  
0           0.420143  
1           0.490950  
2           0.504091  
3           0.419776  
4           0.417368  


In [45]:
# level,structure_id,llm_assignments,query_type,cost,energy,latency,qoa,input_cost,input_latency,input_energy,output_cost,output_latency,output_energy,average_output_tokens


In [8]:
unique_combinations = df[['assignment', 'struct_id', 'Query_Type']].drop_duplicates()
print(unique_combinations)
print("Total unique combinations:", len(unique_combinations))

             assignment  struct_id   Query_Type
0                  (0,)          0          Art
1                  (1,)          0          Art
2                  (2,)          0          Art
3                  (3,)          0          Art
4                  (4,)          0          Art
...                 ...        ...          ...
463445  (3, 3, 3, 3, 5)        840  Video games
463446  (3, 3, 3, 4, 5)        840  Video games
463447  (3, 3, 4, 4, 5)        840  Video games
463448  (3, 4, 4, 4, 5)        840  Video games
463449  (4, 4, 4, 4, 5)        840  Video games

[56400 rows x 3 columns]
Total unique combinations: 56400


In [9]:
exclude_types = ['economics_mmlu', 'engineering_mmlu', 'law_mmlu', 'math_mmlu']
filtered_agg_df = agg_df[~agg_df['Query_Type'].isin(exclude_types)]
print(filtered_agg_df.head())

  assignment  struct_id Query_Type  Total_Cost  Total_Energy  Total_Latency  \
0     (0, 0)          1        Art    0.000009     32.169121      49.514738   
1     (0, 0)          1  Geography    0.000009     30.835237      47.461623   
2     (0, 0)          1    History    0.000009     30.942640      47.626939   
3     (0, 0)          1      Music    0.000008     29.705766      45.723141   
4     (0, 0)          1      Other    0.000009     30.984216      47.690932   

   Quality_of_Answer  
0           0.420143  
1           0.490950  
2           0.504091  
3           0.419776  
4           0.417368  


In [10]:
unique_combinations = filtered_agg_df[['assignment', 'struct_id', 'Query_Type']].drop_duplicates()
print(unique_combinations)
print("Total unique combinations:", len(unique_combinations))

      assignment  struct_id       Query_Type
0         (0, 0)          1              Art
1         (0, 0)          1        Geography
2         (0, 0)          1          History
3         (0, 0)          1            Music
4         (0, 0)          1            Other
...          ...        ...              ...
56393       (4,)          0     history_mmlu
56396       (4,)          0       other_mmlu
56397       (4,)          0  philosophy_mmlu
56398       (4,)          0     physics_mmlu
56399       (4,)          0  psychology_mmlu

[47000 rows x 3 columns]
Total unique combinations: 47000


In [11]:
unique_query_types = filtered_agg_df['Query_Type'].unique()
print(unique_query_types)
print("Total unique query types:", len(unique_query_types))

['Art' 'Geography' 'History' 'Music' 'Other' 'Politics'
 'Science and technology' 'Sports' 'TV shows' 'Video games' 'biology_mmlu'
 'business_mmlu' 'chemistry_mmlu' 'computer science_mmlu' 'health_mmlu'
 'history_mmlu' 'other_mmlu' 'philosophy_mmlu' 'physics_mmlu'
 'psychology_mmlu']
Total unique query types: 20


In [12]:
print(filtered_agg_df.columns.tolist())

['assignment', 'struct_id', 'Query_Type', 'Total_Cost', 'Total_Energy', 'Total_Latency', 'Quality_of_Answer']


In [13]:
# Rename and rearrange columns
filtered_agg_df_renamed = filtered_agg_df.rename(columns={
    'assignment': 'llm_assignments',
    'struct_id': 'structure_id',
    'Query_Type': 'query_type',
    'Total_Cost': 'cost',
    'Total_Energy': 'energy',
    'Total_Latency': 'latency',
    'Quality_of_Answer': 'qoa'
})

# Add 'level' column if needed (set to None or appropriate value)
filtered_agg_df_renamed['level'] = None

# Rearrange columns
filtered_agg_df_final = filtered_agg_df_renamed[['level', 'structure_id', 'llm_assignments', 'query_type', 'cost', 'energy', 'latency', 'qoa']]
print(filtered_agg_df_final.head())

  level  structure_id llm_assignments query_type      cost     energy  \
0  None             1          (0, 0)        Art  0.000009  32.169121   
1  None             1          (0, 0)  Geography  0.000009  30.835237   
2  None             1          (0, 0)    History  0.000009  30.942640   
3  None             1          (0, 0)      Music  0.000008  29.705766   
4  None             1          (0, 0)      Other  0.000009  30.984216   

     latency       qoa  
0  49.514738  0.420143  
1  47.461623  0.490950  
2  47.626939  0.504091  
3  45.723141  0.419776  
4  47.690932  0.417368  


In [14]:
# Map structure_id to level
structure_levels = {
    1: [0, 6, 52, 840],
    2: [1, 50, 802, 836],
    3: [5, 801],
    4: [41]
}

def get_level(structure_id):
    for level, ids in structure_levels.items():
        if structure_id in ids:
            return level
    return None

filtered_agg_df_final['level'] = filtered_agg_df_final['structure_id'].apply(get_level)
print(filtered_agg_df_final.head())

   level  structure_id llm_assignments query_type      cost     energy  \
0      2             1          (0, 0)        Art  0.000009  32.169121   
1      2             1          (0, 0)  Geography  0.000009  30.835237   
2      2             1          (0, 0)    History  0.000009  30.942640   
3      2             1          (0, 0)      Music  0.000008  29.705766   
4      2             1          (0, 0)      Other  0.000009  30.984216   

     latency       qoa  
0  49.514738  0.420143  
1  47.461623  0.490950  
2  47.626939  0.504091  
3  45.723141  0.419776  
4  47.690932  0.417368  


In [15]:
# Add new columns with zero values
for col in ['input_cost', 'input_latency', 'input_energy', 'output_cost', 'output_latency', 'output_energy', 'average_output_tokens']:
    filtered_agg_df_final[col] = 0
print(filtered_agg_df_final.head())

   level  structure_id llm_assignments query_type      cost     energy  \
0      2             1          (0, 0)        Art  0.000009  32.169121   
1      2             1          (0, 0)  Geography  0.000009  30.835237   
2      2             1          (0, 0)    History  0.000009  30.942640   
3      2             1          (0, 0)      Music  0.000008  29.705766   
4      2             1          (0, 0)      Other  0.000009  30.984216   

     latency       qoa  input_cost  input_latency  input_energy  output_cost  \
0  49.514738  0.420143           0              0             0            0   
1  47.461623  0.490950           0              0             0            0   
2  47.626939  0.504091           0              0             0            0   
3  45.723141  0.419776           0              0             0            0   
4  47.690932  0.417368           0              0             0            0   

   output_latency  output_energy  average_output_tokens  
0               

In [16]:
# Print rows where level is 1 and structure_id is 0
print(filtered_agg_df_final[(filtered_agg_df_final['level'] == 1) & (filtered_agg_df_final['structure_id'] == 0)])

       level  structure_id llm_assignments       query_type      cost  \
13200      1             0            (0,)              Art  0.000004   
13201      1             0            (0,)        Geography  0.000004   
13202      1             0            (0,)          History  0.000004   
13203      1             0            (0,)            Music  0.000004   
13204      1             0            (0,)            Other  0.000004   
...      ...           ...             ...              ...       ...   
56393      1             0            (4,)     history_mmlu  0.000022   
56396      1             0            (4,)       other_mmlu  0.000016   
56397      1             0            (4,)  philosophy_mmlu  0.000015   
56398      1             0            (4,)     physics_mmlu  0.000019   
56399      1             0            (4,)  psychology_mmlu  0.000016   

          energy    latency       qoa  input_cost  input_latency  \
13200  14.385158  22.141647  0.380927           0      

In [17]:
fadf = filtered_agg_df_final.copy()

In [18]:
filtered_agg_df_final = fadf.copy()

In [19]:
# ...existing code...

filtered_agg_df_final['average_output_tokens'] = filtered_agg_df_final['average_output_tokens'].astype(float)
filtered_agg_df_final['input_cost'] = filtered_agg_df_final['input_cost'].astype(float)
filtered_agg_df_final['output_cost'] = filtered_agg_df_final['output_cost'].astype(float)
filtered_agg_df_final['input_latency'] = filtered_agg_df_final['input_latency'].astype(float)
filtered_agg_df_final['output_latency'] = filtered_agg_df_final['output_latency'].astype(float)
filtered_agg_df_final['input_energy'] = filtered_agg_df_final['input_energy'].astype(float)
filtered_agg_df_final['output_energy'] = filtered_agg_df_final['output_energy'].astype(float)

cost_map = {
    '(0,)': (2.8e-8, 5.4e-8),
    '(1,)': (5.5e-8, 5.5e-8),
    '(2,)': (6.0e-8, 1.4e-7),
    '(3,)': (6e-8, 2.4e-7),
    '(4,)': (9e-8, 1.6e-7),
}

latency_map = {
    '(0,)': (0.159983, 0.159983),
    '(1,)': (0.15, 0.15),
    '(2,)': (0.133, 0.133),
    '(3,)': (0.09582, 0.09582),
    '(4,)': (0.060, 0.060),
}

energy_map = {
    '(0,)': (0.103939, 0.103939),
    '(1,)': (0.10552, 0.10552),
    '(2,)': (0.1, 0.1),
    '(3,)': (0.067, 0.067),
    '(4,)': (0.083, 0.083),
}

# Example mapping for average_output_tokens per query_type
avg_tokens_map = {
    'Art': 6.517787,
    'Geography': 8.064074,
    'History': 6.678085,
    'Music': 6.352426,
    'Other': 7.421234,
    'Politics': 8.089234,
    'Science and technology': 6.939447,
    'Sports': 8.896979,
    'TV shows': 7.285392,
    'Video games': 8.155149,
    'biology_mmlu': 33.338553,
    'business_mmlu': 45.934000,
    'chemistry_mmlu': 63.100486,
    'computer science_mmlu': 41.072366,
    'economics_mmlu': 30.988043,
    'engineering_mmlu': 39.813846,
    'health_mmlu': 19.795234,
    'history_mmlu': 19.446383,
    'law_mmlu': 27.655660,
    'math_mmlu': 52.694483,
    'other_mmlu': 43.808043,
    'philosophy_mmlu': 16.052809,
    'physics_mmlu': 47.345897,
    'psychology_mmlu': 20.920000
}

print("Updating filtered_agg_df_final with cost, latency, energy, and average output tokens...")
print("Doing cost updates...")
for struct_id, (in_cost, out_cost) in cost_map.items():
    mask = (
        (filtered_agg_df_final['structure_id'] == 0) &
        (filtered_agg_df_final['llm_assignments'] == struct_id)
    )
    filtered_agg_df_final.loc[mask, 'input_cost'] = in_cost
    filtered_agg_df_final.loc[mask, 'output_cost'] = out_cost

print("Doing latency updates...")
for struct_id, (in_cost, out_cost) in latency_map.items():
    mask = (
        (filtered_agg_df_final['structure_id'] == 0) &
        (filtered_agg_df_final['llm_assignments'] == struct_id)
    )
    filtered_agg_df_final.loc[mask, 'input_latency'] = in_cost
    filtered_agg_df_final.loc[mask, 'output_latency'] = out_cost


print("Doing energy updates...")
for struct_id, (in_cost, out_cost) in energy_map.items():
    mask = (
        (filtered_agg_df_final['structure_id'] == 0) &
        (filtered_agg_df_final['llm_assignments'] == struct_id)
    )
    filtered_agg_df_final.loc[mask, 'input_energy'] = in_cost
    filtered_agg_df_final.loc[mask, 'output_energy'] = out_cost


print("Doing average output tokens updates for 0...")
for qtype, avg_tokens in avg_tokens_map.items():
    mask = (
        (filtered_agg_df_final['structure_id'] == 0) &
        (filtered_agg_df_final['llm_assignments'] == '(0,)')
    )
    qtype_mask = mask & (filtered_agg_df_final['query_type'] == qtype)
    filtered_agg_df_final['average_output_tokens'] = filtered_agg_df_final['average_output_tokens'].astype(float)
    filtered_agg_df_final.loc[qtype_mask, 'average_output_tokens'] = avg_tokens


print("Doing average output tokens updates for 1...")
for qtype, avg_tokens in avg_tokens_map.items():
    mask = (
        (filtered_agg_df_final['structure_id'] == 0) &
        (filtered_agg_df_final['llm_assignments'] == '(1,)')
    )
    qtype_mask = mask & (filtered_agg_df_final['query_type'] == qtype)
    filtered_agg_df_final['average_output_tokens'] = filtered_agg_df_final['average_output_tokens'].astype(float)
    filtered_agg_df_final.loc[qtype_mask, 'average_output_tokens'] = avg_tokens

print("Doing average output tokens updates for 2...")
for qtype, avg_tokens in avg_tokens_map.items():
    mask = (
        (filtered_agg_df_final['structure_id'] == 0) &
        (filtered_agg_df_final['llm_assignments'] == '(2,)')
    )
    qtype_mask = mask & (filtered_agg_df_final['query_type'] == qtype)
    filtered_agg_df_final['average_output_tokens'] = filtered_agg_df_final['average_output_tokens'].astype(float)
    filtered_agg_df_final.loc[qtype_mask, 'average_output_tokens'] = avg_tokens

print("Doing average output tokens updates for 3...")
for qtype, avg_tokens in avg_tokens_map.items():
    mask = (
        (filtered_agg_df_final['structure_id'] == 0) &
        (filtered_agg_df_final['llm_assignments'] == '(3,)')
    )
    qtype_mask = mask & (filtered_agg_df_final['query_type'] == qtype)
    filtered_agg_df_final['average_output_tokens'] = filtered_agg_df_final['average_output_tokens'].astype(float)
    filtered_agg_df_final.loc[qtype_mask, 'average_output_tokens'] = avg_tokens

print("Doing average output tokens updates for 4...")
for qtype, avg_tokens in avg_tokens_map.items():
    mask = (
        (filtered_agg_df_final['structure_id'] == 0) &
        (filtered_agg_df_final['llm_assignments'] == '(4,)')
    )
    qtype_mask = mask & (filtered_agg_df_final['query_type'] == qtype)
    filtered_agg_df_final['average_output_tokens'] = filtered_agg_df_final['average_output_tokens'].astype(float)
    filtered_agg_df_final.loc[qtype_mask, 'average_output_tokens'] = avg_tokens


# ...existing code...

Updating filtered_agg_df_final with cost, latency, energy, and average output tokens...
Doing cost updates...
Doing latency updates...
Doing energy updates...
Doing average output tokens updates for 0...
Doing average output tokens updates for 1...
Doing average output tokens updates for 2...
Doing average output tokens updates for 3...
Doing average output tokens updates for 4...


In [22]:
print(len(filtered_agg_df_final[(filtered_agg_df_final['level'] == 1) & (filtered_agg_df_final['structure_id'] == 0)]))

120


In [21]:
# ...existing code...
duplicate_rows = filtered_agg_df_final[(filtered_agg_df_final['structure_id'] == 0) & (filtered_agg_df_final['llm_assignments'] == '(4,)')].copy()
duplicate_rows['llm_assignments'] = '(5,)'
filtered_agg_df_final = pd.concat([filtered_agg_df_final, duplicate_rows], ignore_index=True)
print(f"Appended {len(duplicate_rows)} rows with llm_assignments changed to (5,)")
# ...existing code...

Appended 20 rows with llm_assignments changed to (5,)


In [29]:
level_structure_ids = {
    1: [0, 6, 52, 840],
    2: [1, 50, 802, 836],
    3: [5, 801],
    4: [41]
}
level_dfs = {}
all_ids = []
for level in range(1, 5):
    all_ids += level_structure_ids[level]
    level_dfs[level] = filtered_agg_df_final[filtered_agg_df_final['structure_id'].isin(all_ids)].copy()
    print(f"Level {level} df shape: {level_dfs[level].shape}")


Level 1 df shape: (2520, 15)
Level 2 df shape: (19520, 15)
Level 3 df shape: (34520, 15)
Level 4 df shape: (47020, 15)


In [31]:
for level, df in level_dfs.items():
    fn = f'level_{level}_data.csv'
    df.to_csv(fn, index=False)
    print(f"Saved {fn} ({df.shape[0]} rows, {df.shape[1]} cols)")

Saved level_1_data.csv (2520 rows, 15 cols)
Saved level_2_data.csv (19520 rows, 15 cols)
Saved level_3_data.csv (34520 rows, 15 cols)
Saved level_4_data.csv (47020 rows, 15 cols)


In [25]:
# ...existing code...
level0_df = filtered_agg_df_final[filtered_agg_df_final['structure_id'] == 0].copy()

avg_tokens_map_0 = {
    '(0,)': 420,
    '(1,)': 474,
    '(2,)': 600,
    '(3,)': 454,
    '(4,)': 484,
    '(5,)': 484
}
qoa_map_0 = {
    '(0,)': [0.309, 0.059],
    '(1,)': [0.354, 0.354],
    '(2,)': [0.74, 0.03],
    '(3,)': [0.637, 0.080],
    '(4,)': [0.675, 0.100],
    '(5,)': [0.675, 0.100]
}
for llm_assign, avg_tokens in avg_tokens_map_0.items():
    mask = level0_df['llm_assignments'] == llm_assign
    level0_df.loc[mask, 'average_output_tokens'] = avg_tokens

    qoa_vals = qoa_map_0[llm_assign]
    ends_mmlu = level0_df['query_type'].astype(str).str.endswith('mmlu')

    # use first qoa value for query_types ending with 'mmlu', second value otherwise
    level0_df.loc[mask & ends_mmlu, 'qoa'] = qoa_vals[0]
    level0_df.loc[mask & ~ends_mmlu, 'qoa'] = qoa_vals[1]
print(level0_df.head())
# ...existing code...

       level  structure_id llm_assignments query_type      cost     energy  \
11000      1             0            (0,)        Art  0.000004  14.385158   
11001      1             0            (0,)  Geography  0.000004  13.477424   
11002      1             0            (0,)    History  0.000004  13.667979   
11003      1             0            (0,)      Music  0.000004  12.971587   
11004      1             0            (0,)      Other  0.000004  13.273010   

         latency    qoa    input_cost  input_latency  input_energy  \
11000  22.141647  0.059  2.800000e-08       0.159983      0.103939   
11001  20.744462  0.059  2.800000e-08       0.159983      0.103939   
11002  21.037765  0.059  2.800000e-08       0.159983      0.103939   
11003  19.965878  0.059  2.800000e-08       0.159983      0.103939   
11004  20.429829  0.059  2.800000e-08       0.159983      0.103939   

        output_cost  output_latency  output_energy  average_output_tokens  
11000  5.400000e-08        0.15998

In [26]:
print("Final level 0 df shape:", level0_df.shape)

Final level 0 df shape: (120, 15)


In [32]:
level0_df.to_csv('level_0_data.csv', index=False)

In [24]:
# Print rows where level is 1 and structure_id is 0
print(filtered_agg_df_final[(filtered_agg_df_final['level'] == 1) & (filtered_agg_df_final['structure_id'] == 0)])

       level  structure_id llm_assignments       query_type      cost  \
11000      1             0            (0,)              Art  0.000004   
11001      1             0            (0,)        Geography  0.000004   
11002      1             0            (0,)          History  0.000004   
11003      1             0            (0,)            Music  0.000004   
11004      1             0            (0,)            Other  0.000004   
...      ...           ...             ...              ...       ...   
47015      1             0            (5,)     history_mmlu  0.000022   
47016      1             0            (5,)       other_mmlu  0.000016   
47017      1             0            (5,)  philosophy_mmlu  0.000015   
47018      1             0            (5,)     physics_mmlu  0.000019   
47019      1             0            (5,)  psychology_mmlu  0.000016   

          energy    latency       qoa    input_cost  input_latency  \
11000  14.385158  22.141647  0.380927  2.800000e-08  